In [1]:
import napari
import PIL.Image
import numpy as np 

PIL.Image.MAX_IMAGE_PIXELS = 1056323868

v = napari.Viewer()

# lab = np.zeros((109, 2800, 1987), dtype='int')
# lab[10:90,100:2700,100:1900] = 1
# v.add_labels(lab, scale=(100,3.5,3.5))

Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_0.35_100um//191815_00001_CH1.tif> 

109 total, 54th: <OpenFile 'https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_0.35_100um//191815_05401_CH1.tif'>
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_00001_CH1_resample.tif> 

Meta: {'name': 'mouseID_362188-191815', 'scale': (100, 0.35, 0.35), 'multiscale': True, 'contrast_limits': [0, 65535]}, data: (109, 28001, 19868)
Meta: {'name': 'ilastik mask: 0.5', 'scale': (100, 0.35, 0.35), 'multiscale': True}, data: (109, 28001, 19868)
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_00001_CH1_resample.tif> 

Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.o

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/layers/image/image.py:617: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(


Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_09601_CH1_resample.tif> 
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_00401_CH1_resample.tif> 
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_03801_CH1_resample.tif> 
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_01001_CH1_resample.tif> 
Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_3.5_100um//191815_10201_CH1_resample.tif> 





Reading <File-like object HTTPFileSystem, https://download.brainimagelibrary.org/df/75/df75626840c76c15/mouseID_362188-191815/CH1_

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/layers/image/image.py:617: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(


reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54


/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/layers/image/image.py:617: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/layers/image/image.py:617: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(


reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading label slice 54
reading lab

In [2]:
import h5py 
import numpy as np 
f = h5py.File("/Users/thomasathey/Documents/mimlab/mouselight/BIL/labels/54_slice_2_Probabilities.h5", "r")
pred = f.get("exported_data")
pred = pred[:,:,0]
mask = pred > 0.5
mask = mask.astype('int')
print(np.sum(mask))
mask.dtype


795145


dtype('int64')

In [4]:
v = napari.Viewer()
v.add_labels(mask)

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Labels layer 'mask' at 0x160ed4550>

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari/_vispy/layers/image.py:233: UserWarning: data shape (28001, 19868) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
